In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    return 20 * np.log10(255.0 / np.sqrt(mse))

img = cv2.imread('sar_1.jpg', cv2.IMREAD_GRAYSCALE)

gauss_noise = np.random.normal(0, 25, img.shape).astype(np.uint8)
img_gauss = cv2.add(img, gauss_noise)

uniform_noise = np.random.uniform(-50, 50, img.shape).astype(np.uint8)
img_uniform = cv2.add(img, uniform_noise)

kernels = [3, 5]
sigmas = [1, 2]
d_params = [5, 9]
h_params = [10, 20]

# Списки для хранения лучших результатов
best_results = []

for i in range(2):
    k = kernels[i]
    s = sigmas[i]
    d = d_params[i]
    h = h_params[i]
    
    med_gauss = cv2.medianBlur(img_gauss, k)
    gauss_gauss = cv2.GaussianBlur(img_gauss, (k, k), s)
    bilateral_gauss = cv2.bilateralFilter(img_gauss, d, 75, 75)
    nlm_gauss = cv2.fastNlMeansDenoising(img_gauss, None, h, 7, 21)
    
    med_uniform = cv2.medianBlur(img_uniform, k)
    gauss_uniform = cv2.GaussianBlur(img_uniform, (k, k), s)
    bilateral_uniform = cv2.bilateralFilter(img_uniform, d, 75, 75)
    nlm_uniform = cv2.fastNlMeansDenoising(img_uniform, None, h, 7, 21)
    
    plt.figure(figsize=(15, 5))
    images_gauss = [img, img_gauss, med_gauss, gauss_gauss, bilateral_gauss, nlm_gauss]
    titles_gauss = ['Исходное', 'Гауссов шум', 
                   f'Медианный (k={k})', f'Гаусс (k={k}, s={s})',
                   f'Билатеральный (d={d})', f'NLM (h={h})']
    
    for j in range(6):
        plt.subplot(2, 3, j+1)
        plt.imshow(images_gauss[j], cmap='gray')
        plt.title(titles_gauss[j])
        plt.axis('off')
    
    plt.suptitle(f'Гауссов шум - параметры {i+1}')
    plt.tight_layout()
    plt.show()
    
    # Вычисление PSNR для гауссова шума
    psnr_med_g = psnr(img, med_gauss)
    psnr_gauss_g = psnr(img, gauss_gauss)
    psnr_bilateral_g = psnr(img, bilateral_gauss)
    psnr_nlm_g = psnr(img, nlm_gauss)
    
    print(f"Параметры {i+1} - Гауссов шум:")
    print(f"  Медианный: {psnr_med_g:.2f}")
    print(f"  Гаусс: {psnr_gauss_g:.2f}")
    print(f"  Билатеральный: {psnr_bilateral_g:.2f}")
    print(f"  NLM: {psnr_nlm_g:.2f}")
    
    # Определение лучшего фильтра для гауссова шума
    gauss_results = {
        'Медианный': psnr_med_g,
        'Гаусс': psnr_gauss_g,
        'Билатеральный': psnr_bilateral_g,
        'NLM': psnr_nlm_g
    }
    best_gauss = max(gauss_results, key=gauss_results.get)
    print(f"🎯 ЛУЧШИЙ ФИЛЬТР: {best_gauss} (PSNR: {gauss_results[best_gauss]:.2f} dB)")
    print()
    
    plt.figure(figsize=(15, 5))
    images_uniform = [img, img_uniform, med_uniform, gauss_uniform, bilateral_uniform, nlm_uniform]
    titles_uniform = ['Исходное', 'Равномерный шум', 
                     f'Медианный (k={k})', f'Гаусс (k={k}, s={s})',
                     f'Билатеральный (d={d})', f'NLM (h={h})']
    
    for j in range(6):
        plt.subplot(2, 3, j+1)
        plt.imshow(images_uniform[j], cmap='gray')
        plt.title(titles_uniform[j])
        plt.axis('off')
    
    plt.suptitle(f'Равномерный шум - параметры {i+1}')
    plt.tight_layout()
    plt.show()
    
    # Вычисление PSNR для равномерного шума
    psnr_med_u = psnr(img, med_uniform)
    psnr_gauss_u = psnr(img, gauss_uniform)
    psnr_bilateral_u = psnr(img, bilateral_uniform)
    psnr_nlm_u = psnr(img, nlm_uniform)
    
    print(f"Параметры {i+1} - Равномерный шум:")
    print(f"  Медианный: {psnr_med_u:.2f}")
    print(f"  Гаусс: {psnr_gauss_u:.2f}")
    print(f"  Билатеральный: {psnr_bilateral_u:.2f}")
    print(f"  NLM: {psnr_nlm_u:.2f}")
    
    # Определение лучшего фильтра для равномерного шума
    uniform_results = {
        'Медианный': psnr_med_u,
        'Гаусс': psnr_gauss_u,
        'Билатеральный': psnr_bilateral_u,
        'NLM': psnr_nlm_u
    }
    best_uniform = max(uniform_results, key=uniform_results.get)
    print(f"🎯 ЛУЧШИЙ ФИЛЬТР: {best_uniform} (PSNR: {uniform_results[best_uniform]:.2f} dB)")
    print()
    
    # Сохраняем результаты для итогового анализа
    best_results.append({
        'параметры': i+1,
        'гауссов_шум': {'фильтр': best_gauss, 'psnr': gauss_results[best_gauss]},
        'равномерный_шум': {'фильтр': best_uniform, 'psnr': uniform_results[best_uniform]}
    })

# ИТОГОВЫЙ АНАЛИЗ
print("=" * 70)
print("ИТОГОВЫЙ АНАЛИЗ РЕЗУЛЬТАТОВ")
print("=" * 70)

# Анализ по типам шума
gauss_filters = [result['гауссов_шум']['фильтр'] for result in best_results]
uniform_filters = [result['равномерный_шум']['фильтр'] for result in best_results]

print("📊 СВОДКА ПО ТИПАМ ШУМА:")
print(f"Гауссов шум: {gauss_filters.count('Медианный')}× Медианный, "
      f"{gauss_filters.count('Гаусс')}× Гаусс, "
      f"{gauss_filters.count('Билатеральный')}× Билатеральный, "
      f"{gauss_filters.count('NLM')}× NLM")

print(f"Равномерный шум: {uniform_filters.count('Медианный')}× Медианный, "
      f"{uniform_filters.count('Гаусс')}× Гаусс, "
      f"{uniform_filters.count('Билатеральный')}× Билатеральный, "
      f"{uniform_filters.count('NLM')}× NLM")

# Общий лучший фильтр
all_psnr = []
for result in best_results:
    all_psnr.append((result['гауссов_шум']['фильтр'], result['гауссов_шум']['psnr']))
    all_psnr.append((result['равномерный_шум']['фильтр'], result['равномерный_шум']['psnr']))

best_overall = max(all_psnr, key=lambda x: x[1])
print(f"\n🏆 ОБЩИЙ ЛУЧШИЙ ФИЛЬТР: {best_overall[0]} (PSNR: {best_overall[1]:.2f} dB)")

# Рекомендации
print(f"\n💡 РЕКОМЕНДАЦИИ:")
print(f"• Для гауссова шума лучше использовать: {max(set(gauss_filters), key=gauss_filters.count)}")
print(f"• Для равномерного шума лучше использовать: {max(set(uniform_filters), key=uniform_filters.count)}")
print("=" * 70)

AttributeError: 'NoneType' object has no attribute 'shape'